<a href="https://colab.research.google.com/github/mkbahk/QuantumComputing/blob/main/QML_mnist_example_pennylane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This cell imports all required libraries. We use pytorch to define and train neural networks as well as provide the classical layers. We use torchvision to load the MNIST digits dataset.

In [1]:
!python3 -m pip install pennylane
!python3 -m pip install pennylane-lightning[gpu]
!python3 -m pip install pennylane-qulacs
!python3 -m pip install qulacs
!python3 -m pip install torch
!python3 -m pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.3/870.3 kB 10.7 MB/s eta 0:00:00
ERROR: You must give at least one requirement to install (see "pip help install")


In [2]:
# pytorch dataset loading, model definition, and model training code
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# pennylane imports for defining the quantum part of the model
import pennylane as qml
import pennylane.numpy as np


# for timing the training process
import time

Here we handle loading the MNIST digits and optionally select a reduced set of classes.

In [3]:
# load the MNIST training and testing datasets
training_data = datasets.MNIST(
    root="~/pytorch_data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="~/pytorch_data",
    train=False,
    download=True,
    transform=ToTensor()
)

# allow for restricting classes to specific digits
from functools import reduce
def restrict_classes(dataset, classes):
    classes_membership_mask = reduce(lambda a,b: a|b,
                                     [dataset.targets == i for i in classes],
                                     torch.zeros(dataset.targets.size(), dtype=int))
    idx = torch.where(classes_membership_mask)
    dataset.data = dataset.data[idx]
    dataset.targets = dataset.targets[idx]
    return dataset
###def

# NOTE: you can change this list to select which digit classes are
# used from the datasets.
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
print("Train data:")
print(restrict_classes(training_data, classes))
print()
print("Test data:")
print(restrict_classes(test_data, classes))

100%|██████████| 9912422/9912422 [00:00<00:00, 214922645.94it/s]

Extracting /root/pytorch_data/MNIST/raw/train-images-idx3-ubyte.gz to /root/pytorch_data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 7394438.64it/s]

Extracting /root/pytorch_data/MNIST/raw/train-labels-idx1-ubyte.gz to /root/pytorch_data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 226260923.79it/s]


Extracting /root/pytorch_data/MNIST/raw/t10k-images-idx3-ubyte.gz to /root/pytorch_data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 18513633.40it/s]


Extracting /root/pytorch_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /root/pytorch_data/MNIST/raw

Train data:
Dataset MNIST
    Number of datapoints: 60000
    Root location: /root/pytorch_data
    Split: Train
    StandardTransform
Transform: ToTensor()

Test data:
Dataset MNIST
    Number of datapoints: 10000
    Root location: /root/pytorch_data
    Split: Test
    StandardTransform
Transform: ToTensor()


This cell defines the hybrid model. We first define the quantum part, then the hybrid model class that makes use of it.

In [4]:
# NOTE: here, we configure the hyperparameters of the quantum layer.
# You must install the lightning.qubit device for cpu runs or
# lightning.gpu device for gpu runs. You can also set use_lightning=False
# to use the slow default.qubit backend.
nqubits = 4
nlayers = 2
use_lightning = False
use_gpu = False

# default to cpu pytorch ops
torch_device = "cpu"
if use_lightning:
    if use_gpu:
        qml_device = qml.device('lightning.gpu', wires=nqubits)
        # override to use gpu in pytorch
        torch_device = "cuda"
    else:
        qml_device = qml.device('lightning.qubit', wires=nqubits)
    ###if
else:
    if use_gpu:
        raise RuntimeError("Cannot use gpu without also using lightning simulator.")
    ###if
    qml_device = qml.device('qulacs.simulator', wires=nqubits)
###if

# Here we define the quantum part of the model
@qml.qnode(qml_device, interface="torch")
def qnn_layer(inputs, weights):
    # encode inputs from previous layer
    # as rotations.
    for i in range(nqubits):
        qml.RX(inputs[i], wires=i)
    ###for

    # place gates using the trainable weights
    # as parameters.
    for layer_index in range(nlayers):
        # place the trainable rotations
        for i in range(nqubits):
            qml.RY(weights[i + layer_index * nqubits], wires=i)
        ###for

        # place the entangling gates
        for i in range(nqubits):
            j = (i + 1) % nqubits
            qml.CNOT(wires=(i, j))
        ###for
    ###for

    # now, return the pauli Z expectation values
    # on each qubit.
    return tuple(qml.expval(qml.PauliZ(i)) for i in range(nqubits))
###def

class HybridClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        # to convert image arrays to vectors
        self.flatten = nn.Flatten()
        # to reduce number of features for input to the qnn
        self.reduction_layer = nn.Linear(28*28, nqubits)
        # to map the qnn outputs to a class (some values are unused if
        # not using all 10 classes)
        self.output_layer = nn.Linear(nqubits, 10)

        # This is the randomly initialised weights tensor for the quantum layer.
        self.qnn_weights = torch.rand(nlayers * nqubits) * np.pi
    ###def

    def forward(self, x):
        # transform image array to a vector
        x = self.flatten(x)
        # scale pixel values to [0, 1] range
        x = x / 255.0
        # apply classical dimensionality reduction layer
        x = self.reduction_layer(x)
        # apply pi*tanh activation to put data into the range from -pi
        # to pi
        x = torch.tanh(x) * torch.pi
        # apply the qnn layer to the input and weights.
        # older versions of pennylane do not support batches,
        # so we iterate through the batch and apply the qnn manually.
        batch_size = x.size(0)
        out = torch.empty((batch_size, nqubits), dtype=torch.float)
        for batch_index in range(batch_size):
            expval_tensors = qnn_layer(x[batch_index], self.qnn_weights)
            expval_floats = [t.item() for t in expval_tensors]
            out[batch_index] = torch.tensor(expval_floats)
        ###for

        x = out
        # apply output layer to combine qnn outputs to 10 numbers
        x = self.output_layer(x)
        # just return outputs since we will use cross entropy loss in
        # training
        return x
    ###def
###class

model = HybridClassifier().to(torch_device)

This cell defines function to run single epochs / passes of training and testing.

In [5]:
# NOTE: the below code is all used to train the defined model
# and will be run when this file is loaded.
learning_rate = 1e-3
batch_size = 32
epochs = 30

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(torch_device)
        y = y.to(torch_device)
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}")
        ###if
    ###for
###def


def test_loop(dataloader, model, loss_fn):
    model.eval()
    num_batches = len(dataloader)
    test_loss = 0

    pred = None
    X = None
    correct = 0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(torch_device)
            y = y.to(torch_device)
            pred = model(X)
            predicted_label = torch.argmax(pred, dim=1)
            correct += torch.count_nonzero(predicted_label == y)
            test_loss += loss_fn(pred, y).item()
        ###for
    ###with

    print(f"Accuracy: {correct / (len(dataloader) * batch_size) * 100}")
    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")
    return test_loss
###def

This cell will start the training process when executed.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5)

start_time = time.time()

print(f"Before training\n--------------------------")
test_loop(test_dataloader, model, loss_fn)

for t in range(epochs):
    print(f"Epoch {t+1}\n--------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loss = test_loop(test_dataloader, model, loss_fn)
    scheduler.step(test_loss)
###for

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Training took {elapsed_time/60:.2f} minutes.")

print("Done!")

Before training
--------------------------
Accuracy: 8.905750274658203
Avg loss: 2.365273 

Epoch 1
--------------------------
loss: 2.426853 [   32/60000
loss: 2.358748 [ 3232/60000
loss: 2.291695 [ 6432/60000
loss: 2.334761 [ 9632/60000
loss: 2.347747 [12832/60000
loss: 2.322639 [16032/60000
loss: 2.297662 [19232/60000
loss: 2.303993 [22432/60000
loss: 2.308602 [25632/60000
loss: 2.288624 [28832/60000
loss: 2.310496 [32032/60000
loss: 2.317599 [35232/60000
loss: 2.305528 [38432/60000
loss: 2.302873 [41632/60000
loss: 2.315138 [44832/60000
loss: 2.304488 [48032/60000
loss: 2.304734 [51232/60000
loss: 2.303722 [54432/60000
loss: 2.291229 [57632/60000
Accuracy: 11.331869125366211
Avg loss: 2.301305 

Epoch 2
--------------------------
loss: 2.293177 [   32/60000
loss: 2.296879 [ 3232/60000
loss: 2.306974 [ 6432/60000
loss: 2.286161 [ 9632/60000
loss: 2.307736 [12832/60000
loss: 2.306955 [16032/60000
loss: 2.303900 [19232/60000
loss: 2.297210 [22432/60000
loss: 2.304444 [25632/60000
loss